# Projeto: Análise de Anomalias em Série Temporal

BIMASTER - Trabalho de final de curso

Nome: Alex Marques Campos

Etapa 02.B: Detecção de anomalias via predição de séries

O objetivo deste notebook é carregar os dados das séries históricas de interesse e observar se é possível utilizar predição de séries temporais para realizar a detecção de anomalias. Para isso, um modelo LSTM.

O primeiro passo é carregar as bibliotecas necessárias ao processamento e os dados propriamente ditos, que estão armazenados nos arquivos CSV (_comma separated values_) armazenados no subdiretório './__dados__/'.

## Configuração do ambiente de execução

In [1]:
import math
import matplotlib
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from typing import Tuple

In [2]:
NOME_DIRETORIO_DADOS = 'dados'
NOME_DIRETORIO_MODELOS = 'modelos'
NOME_ARQUIVO_SERIE_IBCBR = 'serie_ibcbr.csv'

In [3]:
print(f'pandas == {pd.__version__}')
print(f'matplotlib == {matplotlib.__version__}')
print(f'tensorflow == {tf.__version__}')

pandas == 1.3.5
matplotlib == 3.2.2
tensorflow == 2.9.2


In [4]:
# ajustamos o formato de apresentação padrão dos gráficos
sns.set_theme(style="white", palette="pastel")

In [5]:
# habilitamos a visualização especial de dataframes disponível no
# Google Colaboratory, para facilitar a exploração dos dados.
from google.colab import data_table
data_table.enable_dataframe_formatter()

## Carga dos dados

Lembre-se de criar uma pasta chamada '/content/dados' no Google Colaboratory e enviar os arquivos '.csv' para essa pasta.

In [6]:
path_dir_dados = Path('.') / NOME_DIRETORIO_DADOS
path_arq_ibcbr = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR

In [7]:
def carregar_csv(path_csv:Path) -> pd.DataFrame:
  """
  Carrega os dados de um arquivo CSV em um dataframe de
  forma padronizada no escopo do projeto.
  path_csv : objeto Path que aponta para o arquivo.
  """
  if (path_csv is None) or (not path_csv.is_file()):
    raise ValueError("The given path object doesn't point to a valid csv file.")

  return pd.read_csv(path_csv,
                   sep=',',
                   parse_dates=True,
                   infer_datetime_format=True,
                   index_col=0,
                   decimal='.',
                   encoding='utf8')

In [8]:
# carregamos o arquivo de dado em um dataframe, para iniciar a análise
df_ibcbr = carregar_csv(path_arq_ibcbr)

In [9]:
def verificar_propriedades(id:int, nome:str, dataframe:pd.DataFrame) -> None:
  """
  Imprime propriedades do dataframe nomeado para inspeção visual dos dados.
  id: inteiro que identifica o dataframe.
  nome: nome do dataframe
  dataframe: objeto do dataframe
  """
  print(f'[{id:03d}] Dataframe: {nome}')
  print('-' * 5)
  print(f'Shape: {dataframe.shape}')
  print('-' * 5)
  print(dataframe.head(4))
  print('')

In [10]:
# verificamos se os dataframes foram carregados corretamente.
series = {
  'IBC-BR': df_ibcbr
}

for idx,serie in enumerate(series.items()):
  verificar_propriedades(idx+1, serie[0], serie[1])

[001] Dataframe: IBC-BR
-----
Shape: (236, 1)
-----
             valor
data              
2003-01-01   96.15
2003-02-01   98.67
2003-03-01  103.41
2003-04-01  102.19



Neste ponto, temos todos os dados carregados em DataFrames pandas e prontos para análise.

## Predição de Série Temporal

### Definição de funções auxiliares

Começamos definindo funções auxiliares ao processo de análise.

In [11]:
def configurar_ticks_anuais(ax):
  '''
  Procedimento auxiliar para configurar a exibição do eixo x
  de um gráfico do matplotlib para dados de séries temporais
  para os quais só sejam marcados os valores ano a ano.
  '''
  ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(base=1))
  ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))

In [12]:
def desenhar_serie_com_anomalias(df:pd.DataFrame,
                                 indice_anomalias:pd.core.indexes.datetimes.DatetimeIndex,
                                 nome_serie:str='IBC-BR') -> None:
  fig = plt.figure(figsize=(16,5))

  plt.plot(df['valor'], 'b', label=f'Índice {nome_serie}')

  for e in indice_anomalias:
    plt.axvline(x=e, color='r', linestyle=':')

  plt.legend(loc='best')
  plt.ylabel(nome_serie)
  plt.xlabel('Tempo')

  # ajustamos o eixo x da figura para exibir ticks a cada ano
  configurar_ticks_anuais(fig.axes[0])

  plt.show()

In [13]:
def calcula_metricas_erro(y_pred,y_test, number_features):
    rmse = math.sqrt(mean_squared_error(y_pred,y_test))
    print('RMSE: ', rmse)

    mse = mean_squared_error(y_pred,y_test)
    print('MSE: ',mse)

    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print('MAPE: ',mape, '%')

    r2 = r2_score(y_pred,y_test)
    print('R2 Score: ', r2)

    if (number_features is not None and number_features > 0):
        number_samples = len(y_pred)
        adj_r2_score = 1-(1-r2)*(number_samples-1)/(number_samples-number_features-1)
        print('R2 Ajustado: ', adj_r2_score)

In [14]:
def montar_treino_teste_validacao_por_janela(dataset:np.array, 
                                             window_size:int=24,
                                             test_size:float=0.15,
                                             val_size:float=0.15,
                                             random_state:int=None
                                             ) -> Tuple[np.array, np.array, np.array, np.array, np.array, np.array]:
    """
    Recebe um dataframe e o divide em conjuntos de treino, validação e teste
    com janelas (dos conjuntos X) de tamanho definido.
    
    Parâmetros:
    dataset      : Numpy array a ser dividido em treino, validação e teste.
    window_size  : Tamanho da janela deslizante a ser aplicada ao dataset e também
                   o tamanho de cada elemento dos conjuntos X.
    test_size    : float entre 0.01 e 1.00, que representa o valor percentual do 
                   conjunto de teste com relação ao tamanho do dataset.
    val_size     : float entre 0.01 e 1.00, que representa o valor percentual do 
                   conjunto de validação com relação ao tamanho do dataset.
    random_state : inteiro que representa o estado randômico da execução do passo
                   da escolha dos elementos de treino e teste.

    Nota: test_size + val_size deve ser um valor menor do que 1.00, pois deve
          existir um conjunto de treino.

    Retorna: X_train, y_train, X_val, y_val, X_test e y_test, onde cada elemento
             é um array numpy.
    """
    data_size = len(dataset)
    X = []
    y = []

    if data_size < (window_size + 1):
      raise ValueError("""
      The input data dimension 0's length should be at least window_size + 1.
      """)

    if test_size < 0.01 or test_size > 1.00:
      raise ValueError('The value of test_size should be between 0.01 and 1.00')

    if val_size < 0.01 or val_size > 1.00:
      raise ValueError('The value of val_size should be between 0.01 and 1.00')
    
    if test_size + val_size >= 1.00:
      raise ValueError('test_size + val_size should be less than 1.00')

    for i in range(window_size, data_size):
        X.append(dataset[i-window_size:i])
        y.append(dataset[i])
    X, y = np.array(X), np.array(y)

    # na primeira chamada, dividimos o conjunto em treino e teste,
    # onde o tamanho do conjunto de teste é explícito (default de 15%) e 
    # o tamanho do conjunto de treino fica proporcional a (1 - test_size) 
    # (default de 85%).
    X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(X, 
                                                                y, 
                                                                test_size=test_size,
                                                                random_state=random_state)

    # para obter o conjunto de validação, dividimos novamente o conjunto de 
    # treino, cujo tamanho agora é diretamente proprorcional a (1 - test_size),
    # em duas partes, mas agora o conjunto de teste será nossa 'validação'.

    # como val_size é referente a 100% e o conjunto agora é menor,
    # recalculamos val_size para o tamanho atual do conjunto de treinamento
    # antes de o dividir novamente.
    val_size_recalculado = val_size / (1 - test_size)
    
    # dividimos novamente o conjunto, agora em treino e validação
    # desabilitamos o shuffle porque ele é desnecessário.
    # passamos o random_state só por completude (como shuffle está desabilitado
    # esse argumento não deveria ter efeito na chamada).
    X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, 
                                                      y_train_tmp, 
                                                      test_size=val_size_recalculado,
                                                      shuffle=False,
                                                      random_state=random_state)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [15]:
def imprimir_propriedades_aux(a_var:np.array, a_name:str) -> None:
  print(f'Tamanho de {a_name} = {len(a_var)}')
  if type(a_var[0]) == np.float64:
    tam_elem = 1
  else:
    tam_elem = len(a_var[0])
  print(f'   Tamanho de um elemento de {a_name} = {tam_elem}')

In [16]:
def imprimir_propriedades(X_train, y_train, X_val, y_val, X_test, y_test) -> None:
  imprimir_propriedades_aux(X_train, 'X_train')
  imprimir_propriedades_aux(y_train, 'y_train')
  imprimir_propriedades_aux(X_val, 'X_val')
  imprimir_propriedades_aux(y_val, 'y_val')
  imprimir_propriedades_aux(X_test, 'X_test')
  imprimir_propriedades_aux(y_test, 'y_test')

### Configuração do estado determinístico do processo

Depois fixamos as sementes randômicas, para que o processo seja previsível a cada execução.

In [17]:
# esse passo é essencial para a reprocibilidade do experimento
random.seed(26)
np.random.seed = 26

In [18]:
SLIDING_WINDOW_SIZE=24
TRAIN_BATCH_SIZE=20
TRAIN_EPOCHS=300

### Normalização e separação dos dados nos conjuntos de treinamento, validação e teste

Separamos os valores dos dados e os índices em dois objetos diferentes, pois não usaremos o índice temporal durante o treinamento.

In [19]:
np_valores = df_ibcbr['valor'].to_numpy()
np_indice = df_ibcbr.index.to_numpy()

In [20]:
print(f'np_valores.shape = {np_valores.shape}')
print(f'np_indice.shape = {np_indice.shape}')

np_valores.shape = (236,)
np_indice.shape = (236,)


In [21]:
# aqui adicionamos uma coluna extra ao array de np_valores
# para poder usar o scaler depois e normalizar os dados.
np_valores_col_extra = np_valores.reshape(-1,1)
np_valores_col_extra.shape

(236, 1)

A seguir, normalizamos os valores para realizar o processamento pelo modelo.

É importante guardarmos o processo de normalização dos dados, para depois poder reconstruir os dados previstos.

In [22]:
# criamos o normalizador (scaler) e o guardamos na variável 'scaler'
scaler = MinMaxScaler()

# treinamos o normalizador e já normalizamos os dados, os armazenado
# na variável 'np_valores_col_extra'
np_valores_col_extra = scaler.fit_transform(np_valores_col_extra)
print(f'dimensões do np_valores_col_extra = {np_valores_col_extra.shape}')

# depois jogamos fora a coluna extra que criamos só para poder utilizar
# o scaler
np_valores_norm = np_valores_col_extra.squeeze()
print(f'dimensões do np_valores_norm = {np_valores_norm.shape}')

dimensões do np_valores_col_extra = (236, 1)
dimensões do np_valores_norm = (236,)


Nesse ponto, temos os dados normalizados e prontos para a divisão nos conjuntos
de treino, validação e teste.

In [23]:
X_train, y_train, X_val, y_val, X_test, y_test = montar_treino_teste_validacao_por_janela(np_valores_norm,
                                                                                          window_size=SLIDING_WINDOW_SIZE)

In [24]:
imprimir_propriedades(X_train, y_train, X_val, y_val, X_test, y_test)

Tamanho de X_train = 148
   Tamanho de um elemento de X_train = 24
Tamanho de y_train = 148
   Tamanho de um elemento de y_train = 1
Tamanho de X_val = 32
   Tamanho de um elemento de X_val = 24
Tamanho de y_val = 32
   Tamanho de um elemento de y_val = 1
Tamanho de X_test = 32
   Tamanho de um elemento de X_test = 24
Tamanho de y_test = 32
   Tamanho de um elemento de y_test = 1


Verificação:

148 (treino) + 32 (validação) + 32 (teste) = 212

O somatório dos elementos nos conjuntos (212) com os 24 primeiros elementos começamos a partir do primeiro elemento de uma janela completa) resulta em 236 (tamanho original). Então está ok.

Os dados também foram embaralhados durante a separação dos conjuntos, mas respeitando a coerencia temporal dentro de cada janela.

### Modelo LSTM

Nossa primeira abordagem será treinar uma rede neural LSTM para fazer predição da série temporal IBC-BR original. Primeiro, para identificar os melhores hiper-parâmetros do modelo, treinaremos uma rede LSTM com todos os dados da série, divididos em janelas de 24 meses.

In [25]:
# input shape do LSTM
n_samples  = len(X_train)
n_input_features = SLIDING_WINDOW_SIZE
n_labels = 1

In [26]:
# definição da rede LSTM
# usamos dropouts para aumentar a resiliência da rede. Evitamos os dropouts recorrentes
# para que a implementação acelerada cuDNN possa ser utilizada.
# documentação em https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM.

model_lstm = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(240, input_shape=(n_input_features, n_labels), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(80, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(24, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1),
])

In [27]:
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 240)           232320    
                                                                 
 dropout (Dropout)           (None, 24, 240)           0         
                                                                 
 lstm_1 (LSTM)               (None, 80)                102720    
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense (Dense)               (None, 24)                1944      
                                                                 
 dropout_2 (Dropout)         (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 2

In [28]:
callback_es = EarlyStopping(monitor='val_loss', 
                            min_delta=0,
                            patience=10,
                            verbose=1,
                            mode='min')

history = model_lstm.fit(x=X_train,
                         y=y_train,
                         validation_data=(X_val, y_val),
                         batch_size=TRAIN_BATCH_SIZE,
                         epochs=TRAIN_EPOCHS,
                         shuffle=False,
                         callbacks =[callback_es])

Epoch 1/300
8/8 [==============================] - 14s 558ms/step - loss: 0.1404 - val_loss: 0.0558
Epoch 2/300
8/8 [==============================] - 1s 181ms/step - loss: 0.0418 - val_loss: 0.0087
Epoch 3/300
8/8 [==============================] - 2s 208ms/step - loss: 0.0319 - val_loss: 0.0200
Epoch 4/300
8/8 [==============================] - 2s 188ms/step - loss: 0.0268 - val_loss: 0.0074
Epoch 5/300
8/8 [==============================] - 1s 124ms/step - loss: 0.0310 - val_loss: 0.0099
Epoch 6/300
8/8 [==============================] - 1s 82ms/step - loss: 0.0237 - val_loss: 0.0066
Epoch 7/300
8/8 [==============================] - 1s 86ms/step - loss: 0.0228 - val_loss: 0.0094
Epoch 8/300
8/8 [==============================] - 1s 102ms/step - loss: 0.0224 - val_loss: 0.0075
Epoch 9/300
8/8 [==============================] - 1s 86ms/step - loss: 0.0202 - val_loss: 0.0069
Epoch 10/300
8/8 [==============================] - 1s 84ms/step - loss: 0.0200 - val_loss: 0.0076
Epoch 11/300